In [1]:
import pandas as pd
import torch
import torchvision
import argparse
import os
import PIL
import utils

from torchvision import datasets, transforms

from main_dino import *
from custom_dataset import GerDataset, remove_element

# Data errors

In [2]:
annot_df = {'img_name' : [],'classes' : [],'x' : [],'y' : [],'width' : [],'height' : []}
empty_list = []
for idx,i in enumerate(os.listdir('kaggle/train/class/')):
    ## Empty Labels
    try:
        tmp_df = pd.read_csv(f'kaggle/train/class/{i}',header=None)
        for j in tmp_df[0].values:
            annot_df['img_name'].append(f'kaggle/train/images/{i.split(".")[0]}.png')
            annot_df['classes'].append(j.split()[0])
            annot_df['x'].append(j.split()[1])
            annot_df['y'].append(j.split()[2])
            annot_df['width'].append(j.split()[3])
            annot_df['height'].append(j.split()[4])
    except pd.errors.EmptyDataError:
        empty_list.append(i)
        continue

## There is 7 images which did not labelled

In [3]:
empty_list

['07081.txt',
 '02528.txt',
 '04966.txt',
 '06229.txt',
 '04285.txt',
 '03798.txt',
 '02448.txt']

In [4]:
pd.read_csv('kaggle/train/class/00863.txt')

,0 0.633740 0.227441 0.103418 0.109180
0,0 0.185840 0.520020 0.106836 0.116211
1,0 0.045996 0.611328 0.091992 0.126953
2,0 0.018457 0.129932 0.036914 0.107715


## Check custom torch dataset

In [5]:
#data = GerDataset('kaggle/train')

## Train Model

In [6]:
!python -m torch.distributed.launch main_dino.py --batch_size_per_gpu 16 --num_workers 10

/opt/conda/lib/python3.8/site-packages/torch/distributed/launch.py:178: FutureWarning: The module torch.distributed.launch is deprecated
and will be removed in future. Use torchrun.
Note that --use_env is set by default in torchrun.
If your script expects `--local_rank` argument to be set, please
change it to read from `os.environ['LOCAL_RANK']` instead. See 
https://pytorch.org/docs/stable/distributed.html#launch-utility for 
further instructions

  warnings.warn(
Using cache found in /home/vscode/.cache/torch/hub/facebookresearch_xcit_main
| distributed init (rank 0): env://
git:
  sha: 54b0123fa6fbe47a23fdb5ee8053044dd0acf94a, status: has uncommited changes, branch: main

arch: vit_base
batch_size_per_gpu: 16
clip_grad: 3.0
data_path: kaggle/train
dist_url: env://
drop_path_rate: 0.1
epochs: 100
freeze_last_layer: 1
global_crops_scale: (0.4, 1.0)
gpu: 0
local_crops_number: 8
local_crops_scale: (0.05, 0.4)
local_rank: 0
lr: 0.0005
min_lr: 1e-06
momentum_teacher: 0.996
norm_last_layer